# **Projet 5 :** Segmentez des clients d'un site e-commerce

## Notebook de nettoyage et d'ingestion des données

### Imports

In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode

### Configuration

In [2]:
%load_ext autoreload
%autoreload 2

# configurations pandas
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)

### Chargement et nettoyage des données **geolocation**

In [3]:
# GEOLOCATION

df_geo = pd.read_csv("./data/olist_geolocation_dataset.csv", sep=',', header=0)
df_geo.info()
df_geo.drop_duplicates(inplace=True)

# On normalize autant que possible le texte
df_geo['geolocation_city'] = df_geo['geolocation_city'].apply(lambda x : unidecode(str(x).lower()))
df_geo['geolocation_state'] = df_geo['geolocation_state'].apply(lambda x : unidecode(str(x).lower()))

# On ne garde que un unique zipcode car les customers ne sont catégorisés que par leurs zipcode
df_geo_lite = df_geo.drop(columns=['geolocation_lat', 'geolocation_lng'], axis=1)
df_geo_lite.drop_duplicates(subset=['geolocation_zip_code_prefix'], keep='first', inplace=True)

print(df_geo_lite.info())

# Save lite df
file = './data/olist_geolocation_dataset_lite.csv'
with open(file, 'wb+') as f:
    _ = df_geo_lite.to_csv(f, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19015 entries, 0 to 999846
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   geolocation_zip_code_prefix  19015 non-null  int64 
 1   geolocation_city             19015 non-null  object
 2   geolocation_state            19015 non-null  object
dtypes: int

### Chargement et nettoyage des données **customers**

In [4]:
# CUSTOMERS

df_customers = pd.read_csv("./data/olist_customers_dataset.csv", sep=',', header=0)
df_customers.info()
df_customers.drop_duplicates(inplace=True)

# On normalize autant que possible le texte
df_customers['customer_city'] = df_customers['customer_city'].apply(lambda x : unidecode(str(x).lower()))
df_customers['customer_state'] = df_customers['customer_state'].apply(lambda x : unidecode(str(x).lower()))

# On drop "city" et "state" car redondent
df_customers_lite = df_customers.drop(columns=['customer_city', 'customer_state', 'customer_id'], axis=1).drop_duplicates()

nb_duplicated = df_customers_lite[df_customers_lite.duplicated(subset=['customer_unique_id'], keep='first') == True].shape[0]
print(f"{nb_duplicated} 'customer_unique_id' dupliqué !\n")
print(df_customers_lite.info())

df_customers_lite.drop_duplicates(subset=['customer_unique_id'], keep='first', inplace=True)

# Save lite df
file = './data/olist_customers_dataset_lite.csv'
with open(file, 'wb+') as f:
    _ = df_customers_lite.to_csv(f, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
254 'customer_unique_id' dupliqué !

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96350 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_unique_id        96350 non-null  object
 1   customer_zip_code_prefix  96350 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.2+ MB
None


### Chargement et nettoyage des données **orders**

In [5]:
# ORDERS

df_orders = pd.read_csv("./data/olist_orders_dataset.csv", sep=',', header=0, parse_dates=['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'])
df_orders.info()
df_orders.drop_duplicates(inplace=True)

# On normalize autant que possible le texte
df_orders['order_status'] = df_orders['order_status'].apply(lambda x : unidecode(str(x).lower()))

# On merge le "customer_unique_id" sur le "customer_id"
df_orders = df_orders.merge(df_customers[['customer_id', 'customer_unique_id']], left_on='customer_id', right_on='customer_id')

df_orders_lite = df_orders.drop(columns=['customer_id'], axis=1)

print(df_orders_lite.info())

# Save lite df
file = './data/olist_orders_dataset_lite.csv'
with open(file, 'wb+') as f:
    _ = df_orders_lite.to_csv(f, index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dty

### Chargement et nettoyage des données **sellers**

In [6]:
# SELLERS

df_sellers = pd.read_csv("./data/olist_sellers_dataset.csv", sep=',', header=0)
df_sellers.info()
df_sellers.drop_duplicates(inplace=True)

# On normalize autant que possible le texte
df_sellers['seller_city'] = df_sellers['seller_city'].apply(lambda x : unidecode(str(x).lower()))
df_sellers['seller_state'] = df_sellers['seller_state'].apply(lambda x : unidecode(str(x).lower()))

# On drop "city" et "state" car redondent
df_sellers_lite = df_sellers.drop(columns=['seller_city', 'seller_state'], axis=1)

print(df_sellers_lite.info())

# Save lite df
file = './data/olist_sellers_dataset_lite.csv'
with open(file, 'wb+') as f:
    _ = df_sellers_lite.to_csv(f, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3095 entries, 0 to 3094
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 72.5+ KB
None


### Chargement et nettoyage des données **reviews**

In [7]:
# ORDER REVIEWS

df_reviews = pd.read_csv("./data/olist_order_reviews_dataset.csv", sep=',', header=0, parse_dates=['review_creation_date', 'review_answer_timestamp'])
df_reviews.info()
df_reviews.drop_duplicates(inplace=True)

# On normalize autant que possible le texte
df_reviews_update = df_reviews.copy()
df_reviews_update['review_comment_title'] = df_reviews['review_comment_title'].apply(lambda x : unidecode(str(x).lower()))
df_reviews_update['review_comment_message'] = df_reviews['review_comment_message'].apply(lambda x : unidecode(str(x).lower()))

df_reviews_update.info()

# Save lite df
file = './data/olist_order_reviews_dataset_update.csv'
with open(file, 'wb+') as f:
    _ = df_reviews_update.to_csv(f, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                99224 non-null  object        
 1   order_id                 99224 non-null  object        
 2   review_score             99224 non-null  int64         
 3   review_comment_title     11568 non-null  object        
 4   review_comment_message   40977 non-null  object        
 5   review_creation_date     99224 non-null  datetime64[ns]
 6   review_answer_timestamp  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 5.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                99224 non-null

### Chargement et nettoyage des données **items**

In [8]:
# ORDER ITEMS

df_items = pd.read_csv("./data/olist_order_items_dataset.csv", sep=',', header=0, parse_dates=['shipping_limit_date'])
df_items.info()
df_items.drop_duplicates(inplace=True)

df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1 

### Chargement et nettoyage des données **payments**

In [9]:
# ORDER PAYMENTS

df_payments = pd.read_csv("./data/olist_order_payments_dataset.csv", sep=',', header=0)
df_payments.info()
df_payments.drop_duplicates(inplace=True)

df_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null

### Chargement et nettoyage des données **products**

In [10]:
# ORDER PRODUCTS

df_products = pd.read_csv("./data/olist_products_dataset.csv", sep=',', header=0)
df_products.info()
df_products.drop_duplicates(inplace=True)

df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      -------------- 

### Creation des tables sqlite avec les modèles sqlalchemy

In [11]:
from loguru import logger
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from unidecode import unidecode

import model

DATABASE_URL = 'sqlite:///./db/olist.db'

def init_db(uri):
    engine = create_engine(uri)
    db_session = scoped_session(sessionmaker(autocommit=False, autoflush=False, bind=engine))
    model.Base.query = db_session.query_property()
    model.Base.metadata.drop_all(bind=engine)
    model.Base.metadata.create_all(bind=engine)
    return db_session

def insert_record(csv, model_class, session, parse_dates=False):
    try:
        data = pd.read_csv(csv, sep=',', header=0, parse_dates=parse_dates)
        for row_dict in data.replace({np.nan: None}).to_dict(orient="records"):
            record = model_class(**row_dict)
            session.add(record)
        session.commit()
    except Exception as error:
        logger.exception(error)
        session.rollback()
        return
    logger.info(f"[{model_class.__name__}] OK")

In [12]:
# inserssion dans une base de donnée relationnel
session = init_db(DATABASE_URL)
insert_record("./data/olist_geolocation_dataset_lite.csv", model.Geolocation, session)
insert_record("./data/olist_customers_dataset_lite.csv", model.Customers, session)
insert_record("./data/olist_sellers_dataset_lite.csv", model.Sellers, session)
insert_record("./data/olist_orders_dataset_lite.csv", model.Orders, session,
              parse_dates=['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'])
insert_record("./data/olist_order_reviews_dataset_update.csv", model.OrderReviews, session, parse_dates=['review_creation_date', 'review_answer_timestamp'])
insert_record("./data/olist_order_items_dataset.csv", model.OrderItems, session, parse_dates=['shipping_limit_date'])
insert_record("./data/olist_order_payments_dataset.csv", model.OrderPayments, session)
insert_record("./data/olist_products_dataset.csv", model.Products, session)

2022-03-24 15:28:11.592 | INFO     | __main__:insert_record:29 - [Geolocation] OK
2022-03-24 15:28:18.074 | INFO     | __main__:insert_record:29 - [Customers] OK
2022-03-24 15:28:18.247 | INFO     | __main__:insert_record:29 - [Sellers] OK
2022-03-24 15:28:30.027 | INFO     | __main__:insert_record:29 - [Orders] OK
2022-03-24 15:28:39.847 | INFO     | __main__:insert_record:29 - [OrderReviews] OK
2022-03-24 15:28:48.986 | INFO     | __main__:insert_record:29 - [OrderItems] OK
2022-03-24 15:28:55.951 | INFO     | __main__:insert_record:29 - [OrderPayments] OK
2022-03-24 15:28:58.480 | INFO     | __main__:insert_record:29 - [Products] OK


### Export du notebook en HTML

In [1]:
!jupyter nbconvert --config nbconvert/config_html.py

[NbConvertApp] Converting notebook P5_01_notebooknettoyage.ipynb to html
[NbConvertApp] Writing 614238 bytes to P5_01_notebooknettoyage.html
[NbConvertApp] Converting notebook P5_02_notebookanalyse.ipynb to html
[NbConvertApp] Writing 19632329 bytes to P5_02_notebookanalyse.html
[NbConvertApp] Converting notebook P5_03_notebookproduction.ipynb to html
[NbConvertApp] Writing 910310 bytes to P5_03_notebookproduction.html
